# Environment Setup & Authentication

Welcome to the project onboarding notebook. This notebook helps you configure your local environment and validate access to required services such as Google Cloud (BigQuery, Cloud Storage).

One key step in this setup involves authenticating with Google Cloud using a **Service Account**. Each teammate will need access to a JSON key file for the project's service account that authenticates their access to shared cloud resources.

Your service account key file should be placed in the credentials folder. This keeps sensitive files organized and makes it easier to manage your environment setup across machines or users.

> **Important:** Never commit your service account JSON file to version control. The `.gitignore` includes all file found within the `credentials/` directory so `.env` files and the JSON key files will not be pushed to the public repo.

The next section will create and validate a `.env` file that stores the path to your service account credentials and confirms successful authentication.


## GCP Authentication & `.env` Setup

This code block does the following:

1. Checks whether `credentials/secrets.env` exists.
2. If missing, it creates a **template** with a placeholder for your service account key.
3. It then attempts to load the environment variable `GOOGLE_APPLICATION_CREDENTIALS` from the file.
4. If a valid path is found and the file exists, it initializes your GCP clients (BigQuery, Cloud Storage) and prints your authenticated service account email.

> If the `.env` file is missing, the script will create it and **halt execution**, allowing you to add your credentials before continuing. Once the `.env` file is created, add the full path to your JSON key file which should also be stored in the `credentials/` directory.

Once authenticated, you can begin querying BigQuery or interacting with GCS buckets programmatically.


In [1]:
import sys
from dotenv import load_dotenv, find_dotenv
import os
from pathlib import Path
sys.path.append(str(Path("..").resolve()))

from google.cloud import storage, bigquery
from google.auth import default
from data_pipeline.uploader import DataUploader
from data_acquisition.loader import main as run_loader
from data_acquisition.data_cleaner import main as run_cleaner


In [2]:
# GCP Authentication & `.env` Setup
# This script sets up Google Cloud authentication and checks for the necessary environment variables.

# Define secrets file path
secrets_path = Path("../credentials/secrets.env")

# Create file if it doesn't exist
if not secrets_path.exists():
    print("'secrets.env' not found. Creating a template...")
    secrets_path.parent.mkdir(parents=True, exist_ok=True)
    secrets_path.write_text("GOOGLE_APPLICATION_CREDENTIALS=path/to/your/service_account.json\n")
    print(f"Created template at: {secrets_path.resolve()}")
    print("Please update this file with the directory path to your GCP JSON key.")
    print("Store JSON key in the 'credentials' directory to prevent upload to GitHub.")
    sys.exit(1)  
else:
    print(f"Found existing secrets file at: {secrets_path.resolve()}")
    
load_dotenv(find_dotenv("../credentials/secrets.env"))

cred_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

if not cred_path or not os.path.exists(cred_path):
    print(
        "GOOGLE_APPLICATION_CREDENTIALS is not set or the file does not exist.\n"
        "Please ensure secrets.env contains a valid path to your service account JSON file."
    )
    storage_client = None
    bq_client = None
else:
    print("GOOGLE_APPLICATION_CREDENTIALS loaded from .env")

    # Initialize GCP clients using ADC
    storage_client = storage.Client()
    bq_client = bigquery.Client()

    # Confirm authentication
    creds, project_id = default()
    member_email = creds.service_account_email
    print(f"Authenticated as: {member_email}")
    print(f"GCP Project ID: {project_id}")

# GCP configuration
REGION = "us-east1"
print(f"GCP region set to: {REGION}")


Found existing secrets file at: C:\Users\iauge\Documents\Drexel MSDS\DSCI 591\DSCI591-FACTS\credentials\secrets.env
GOOGLE_APPLICATION_CREDENTIALS loaded from .env
Authenticated as: 13742792432-compute@developer.gserviceaccount.com
GCP Project ID: dsci-591-capstone
GCP region set to: us-east1


In [3]:

# Initialize GCP clients with the project ID if provided
storage_client = storage.Client(project = project_id if project_id else None)
bq_client = bigquery.Client(project = project_id if project_id else None)

In [ ]:
# Download raw data files from URLs and Hugging Face
run_loader(force=False, prompt_user=False)

In [4]:
# Convert JSON to JSONL
# Clean JSONL data structure for BigQuery upload
# Clean CSV data structure for BigQuery upload

run_cleaner()


Cleaning datasets: 0it [00:00, ?it/s]

Processing JSONL file: fever_dev_train.jsonl


Cleaning fever_dev_train.jsonl: 19998it [00:00, 64521.70it/s]
Cleaning datasets: 1it [00:00,  3.03it/s]

Converting hotpot_dev_distractor.json to JSONL format


Cleaning datasets: 1it [00:01,  3.03it/s]

...

Processing CSV file: squad_v2_train.csv
Cleaning SQuAD CSV: squad_v2_train.csv


Cleaning datasets: 10it [00:57,  6.64s/it]

Saved cleaned CSV to: ..\data\clean\squad_v2_train.csv
Saved 109 failed rows to: ..\data\raw\squad_v2_train_failed.csv
Processing CSV file: squad_v2_train_failed.csv
Cleaning SQuAD CSV: squad_v2_train_failed.csv
Saved cleaned CSV to: ..\data\clean\squad_v2_train_failed.csv
Saved 109 failed rows to: ..\data\raw\squad_v2_train_failed_failed.csv
Processing CSV file: squad_v2_validation.csv
Cleaning SQuAD CSV: squad_v2_validation.csv


Cleaning datasets: 16it [00:58,  3.66s/it]

Saved cleaned CSV to: ..\data\clean\squad_v2_validation.csv
Saved 11 failed rows to: ..\data\raw\squad_v2_validation_failed.csv
Processing CSV file: squad_v2_validation_failed.csv
Cleaning SQuAD CSV: squad_v2_validation_failed.csv
Saved cleaned CSV to: ..\data\clean\squad_v2_validation_failed.csv
Saved 11 failed rows to: ..\data\raw\squad_v2_validation_failed_failed.csv
Processing CSV file: truthful_qa_train.csv
No specific cleaning function for truthful_qa_train.csv, copying as is.
Data cleaning completed successfully.


In [5]:
# Initialize DataUploader with BigQuery client and project ID
uploader = DataUploader(
    bq_client=bq_client,
    project_id=project_id,
    dataset_name="data_clean"
)

# Upload files to BigQuery
for path in Path("../data/clean").glob("*.*"):
    name = path.stem
    uploader.upload_to_bigquery(
        file_path=path,
        table_name=name
    )


Table dsci-591-capstone.data_clean.fever_dev_train already exists. Using existing table.
Successfully uploaded ..\data\clean\fever_dev_train.jsonl to BigQuery table dsci-591-capstone.data_clean.fever_dev_train.
Table dsci-591-capstone.data_clean.hotpot_dev_distractor already exists. Using existing table.
Successfully uploaded ..\data\clean\hotpot_dev_distractor.jsonl to BigQuery table dsci-591-capstone.data_clean.hotpot_dev_distractor.
Table dsci-591-capstone.data_clean.hotpot_dev_fullwiki already exists. Using existing table.
Successfully uploaded ..\data\clean\hotpot_dev_fullwiki.jsonl to BigQuery table dsci-591-capstone.data_clean.hotpot_dev_fullwiki.
Table dsci-591-capstone.data_clean.hotpot_train already exists. Using existing table.
Successfully uploaded ..\data\clean\hotpot_train.jsonl to BigQuery table dsci-591-capstone.data_clean.hotpot_train.
Table dsci-591-capstone.data_clean.nq_open_train already exists. Using existing table.
Successfully uploaded ..\data\clean\nq_open_trai